<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://assets.vespa.ai/logos/Vespa-logo-green-RGB.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://assets.vespa.ai/logos/Vespa-logo-dark-RGB.svg">
  <img alt="#Vespa" width="200" src="https://assets.vespa.ai/logos/Vespa-logo-dark-RGB.svg" style="margin-bottom: 25px;">
</picture>


# Video Search and Retrieval with Vespa and TwelveLabs

In the following notebook, we will demonstrate how to leverage [TwelveLabs](https://www.twelvelabs.io/) `Marengo-retrieval-2.7` a SOTA multimodal embedding model to demonstrate a use case of video embeddings storage and semantic search retrieval using Vespa.ai.

The steps we will take in this notebook are:

1. Setup and configuration
2. Generate Attributes and Embeddings for 3 sample videos using the TwelveLabs python SDK.
3. Deploy the Vespa application to Vespa Cloud and Feed the Data
4. Perform a semantic search with hybrid multi-phase ranking on the videos
5. Review the results

All the steps that are needed to provision the Vespa application, including feeding the data, can be done by running this notebook.
We have tried to make it easy for others to run this notebook, to create your own Video semantic search application using TwelveLabs models with Vespa.

## 1. Setup and Configuration

For reference, this is the Python version used for this notebook.

In [ ]:
!python --version

Python 3.11.11


### 1.1 Install libraries

Install the required Python dependencies from TwelveLabs python SDK and pyvespa python API.

In [ ]:
!pip3 install twelvelabs
!pip3 install pyvespa vespacli

Import all the required packages in this notebook.

In [ ]:
import os
import hashlib
import requests
import json

from vespa.package import (
    ApplicationPackage,
    Field,
    Schema,
    Document,
    HNSW,
    RankProfile,
    Component,
    Parameter,
    FieldSet,
    SecondPhaseRanking,
    Function
)

from vespa.deployment import VespaCloud
from vespa.io import VespaResponse, VespaQueryResponse

from twelvelabs import TwelveLabs
from typing import List
from twelvelabs.models.embed import EmbeddingsTask, SegmentEmbedding

from itertools import islice
import pandas as pd


### 1.2 Get a TwelveLabs API key
[Sign-up](https://auth.twelvelabs.io/u/signup) for TwelveLabs.

After logging in, navigate to your profile and get your [API key](https://playground.twelvelabs.io/dashboard/api-key). Copy it and paste it below.

The Free plan includes indexing of 600 mins of videos, which should be sufficient to explore the capabilities of the API.

In [ ]:
TL_API_KEY = os.getenv("TL_API_KEY") or input(
    "Enter your TL_API key: "
)

###1.3 Sign-up for a Vespa Trial Account

**Pre-requisite**:
- Spin-up a Vespa Cloud [Trial](https://vespa.ai/free-trial) account.
- Login to the account you just created and create a tenant at [console.vespa-cloud.com](https://console.vespa-cloud.com/).
- Save the tenant name.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xxx)

###1.4 Setup the tenant name and the application name

- Paste below the name of the tenant name.
- Give your application a name. Note that the name cannot have `-` or `_`.

In [ ]:
# Replace with your tenant name from the Vespa Cloud Console
tenant_name = "vespa-acme"
# Replace with your application name (does not need to exist yet)
application = "twelvelabsmv"

## 2. Generate Attributes and Embeddings for sample videos using TwelveLabs Embedding API



### 2.1 Generate attributes on the videos

In this section, we will leverage the [Pegasus 1.1](https://docs.twelvelabs.io/docs/video-understanding-models) generative model to generate some attributes about our videos to store as part of the searchable information in Vespa. Attributes we want to store as part of the videos include:

- Keywords
- Summaries

For video samples, we are selecting the 3 videos in the array below from the [Internet Archive](https://archive.org/).

You can customize this code with the urls of your choice. Note that there are certain restrictions such as the resolution of the videos.


In [ ]:
VIDEO_URLs = [
               "https://ia801503.us.archive.org/27/items/hide-and-seek-with-giant-jenny/HnVideoEditor_2022_10_29_205557707.ia.mp4",
               "https://ia601401.us.archive.org/1/items/twas-the-night-before-christmas-1974-full-movie-freedownloadvideo.net/twas-the-night-before-christmas-1974-full-movie-freedownloadvideo.net.mp4",
               "https://dn720401.ca.archive.org/0/items/mr-bean-the-animated-series-holiday-for-teddy/S2E12.ia.mp4"]

In order to generate text on the videos, the prerequisite is to upload the videos and index them. Let's first create an index below:

In [ ]:
from glob import glob
# The path to the directory containing the videos you wish to upload.
client = TwelveLabs(api_key=TL_API_KEY)

index = client.index.create(
    name="Vespa-Generate-1",
    models=[
        {
            "name": "pegasus1.1",
            "options": ["visual", "audio"],
        }
    ],
    addons=["thumbnail"], # Optional
)
print(f"Created index: id={index.id} name={index.name} models={index.models}")


Created index: id=67896b37c6dde497bd15fc7d name=Vespa-Generate-1 models=root=[Model(name='pegasus1.1', options=['visual', 'audio'], addons=None, finetuned=False)]


We can now upload the videos:

In [ ]:
index_id = index.id

def on_task_update(task: EmbeddingsTask):
    print(f"  Status={task.status}")


for video_url in VIDEO_URLs:
  # Create a video indexing task
  task = client.task.create(index_id=index_id, url=video_url, language="en")
  print(f"Task created successfully! Task ID: {task.id}")
  status = task.wait_for_done(
  sleep_interval=10,
  callback=on_task_update
  )
  print(f"Indexing done: {status}")
  if task.status != "ready":
      raise RuntimeError(f"Indexing failed with status {task.status}")
  print(f"Uploaded {video_url}. The unique identifer of your video is {task.video_id}.")



Task created successfully! Task ID: 67896b41a761bcb96501c235
  Status=pending
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=indexing
  Status=ready
Indexing done: Task(id='67896b41a761bcb96501c235', created_at='2025-01-16T20:25:37.121Z', updated_at='2025-01-16T20:25:37.121Z', index_id='67896b37c6dde497bd15fc7d', video_id='67896b42027eec4fa50e83b4', status='ready', system_metadata={'filename': 'HnVideoEditor_2022_10_29_205557707.ia', 'duration': 221.9666671, 'width': 854, 'height': 480}, hls=None)
Uploaded https://ia801503.us.archive.org/27/items/hide-and-seek-with-giant-jenny/HnVideoEditor_2022_10_29_205557707.ia.mp4. The unique identifer of your video is 67896b42027eec4fa50e83b4.
Task created successfully! Task ID: 67896bdec6dde497bd15fc7e
  Status=pending
  Status=pending
  Status=pending
  Status=indexing
 

Now that the videos have been uploaded, we can generate the keywords, and summaries on the videos below. You will notice on the output that the video uploaded last is the one that is processed first in this stage. This matters since we store other attributes on the videos on arrays (eg URLs, Titles).

In [ ]:
client = TwelveLabs(api_key=TL_API_KEY)

summaries = []
keywords_array = []

#Get all videos in an Index
videos = client.index.video.list(index_id)
for video in videos:
    print(f"Generating text for {video.id}")

    res = client.generate.summarize(video_id=video.id, type="summary")
    print(f"Summary: {res.summary}")
    summaries.append(res.summary)

    keywords = client.generate.text(video_id=video.id, prompt="Based on this video, I want to generate five keywords for SEO (Search Engine Optimization). Provide just the keywords as a comma delimited list without any additional text.")
    print(f"Open-ended Text: {keywords.data}")
    keywords_array.append(keywords.data)

Generating text for 67896cee027eec4fa50e83b5
Summary: "Mr. Bean: Holiday for Teachers" is a delightful animated cartoon that captures the whimsical and humorous essence of the beloved character, Mr. Bean. The story begins with Mr. Bean in a classroom setting, where he humorously introduces the concept of a holiday for teachers. His antics, including accidentally dropping a globe on a student's head and using his teddy bear as a teaching prop, set the tone for the lighthearted adventure that follows.
Deciding to take a holiday himself, Mr. Bean embarks on a journey with his trusty teddy bear. Along the way, he faces various challenges, such as losing his luggage and getting caught in a rainstorm. Despite these setbacks, Mr. Bean's optimism and determination shine through, providing plenty of comedic moments. The animation style is simple yet effective, capturing the charm of the original live-action series.
The narrative then shifts to a series of animated scenes featuring different cha

We need to store the titles of the videos as an additional attribute. We also want to overwrite the summary for one of the videos as we are not getting an accurate summary for one of the videos.

In [ ]:
#Creating array with titles
titles=["Mr. Bean the Animated Series Holiday for Teddy","Twas the night before Christmas", "Hide and Seek with Giant Jenny"]

#Overwrite summary due to hallucination
summaries[1] = "In Junctionville, Santa cancels Christmas after receiving an anonymous letter claiming he doesn’t exist. Joshua Trundle and Father Mouse create a special clock tower to win Santa back. Albert Mouse accidentally breaks the clock mechanism, causing public embarrassment. Feeling remorseful, Albert repairs the clock just before Christmas Eve. The clock’s magical song convinces Santa to return and restore Christmas."""

## 2.2 Generate Embeddings

The following code leverages the [Embed API](https://docs.twelvelabs.io/docs/create-video-embeddings) to create an asynchronous embedding task to embed the sample videos.

Twelve Labs video embeddings capture all the subtle cues and interactions between different modalities, including the visual expressions, body language, spoken words, and the overall context of the video, encapsulating the essence of all these modalities and their interrelations over time.



In [ ]:
client = TwelveLabs(api_key=TL_API_KEY)

# Initialize an array to store the task IDs as strings
task_ids = []

for url in VIDEO_URLs:
  task = client.embed.task.create(
    model_name="Marengo-retrieval-2.7",
    video_url=url)
  print(f"Created task: id={task.id} model_name={task.model_name} status={task.status}")
  # Append the task ID to the array
  task_ids.append(str(task.id))
  status = task.wait_for_done(
  sleep_interval=10,
  callback=on_task_update
  )
  print(f"Embedding done: {status}")
  if task.status != "ready":
      raise RuntimeError(f"Embedding failed with status {task.status}")

Created task: id=67896e59c6dde497bd15fc8b model_name=Marengo-retrieval-2.7 status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=ready
Embedding done: ready
Created task: id=67896ea6c6dde497bd15fc91 model_name=Marengo-retrieval-2.7 status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processi

## 2.3 Retrieve Embeddings

Once the embedding task is completed, we can retrieve the results of the embedding task based on the task_ids.

In [ ]:
# Spin-up session
client = TwelveLabs(api_key=TL_API_KEY)

# Initialize an array to store the task objects directly
tasks = []

for task_id in task_ids:
    # Retrieve the task
    task = client.embed.task.retrieve(task_id)
    tasks.append(task)

    # Print task details
    print(f"Task ID: {task.id}")
    print(f"Status: {task.status}")

Task ID: 67896e59c6dde497bd15fc8b
Status: ready
Task ID: 67896ea6c6dde497bd15fc91
Status: ready
Task ID: 67896f6ba761bcb96501c241
Status: ready


We can now review the output structure of the first segment for each one of these videos. This output will help us define the schema to store the embeddings in Vespa in the second part of this notebook.

From looking at this output, the video has been embedded into chunks of 6 seconds each (default configurable value in the Embed API). Each embedding has a float vector of dimension 1024.

The number of segments generated vary per video, based on the length of the videos ranging from 37 to 242 segments.

In [ ]:
for task in tasks:
  print(task.id)
  # Display data types of each field
  for key, value in task.video_embedding.segments[0]:
   if isinstance(value, list):
        print(f"{key}: list of size {len(value)} (truncated to 5 items): {value[:5]} ")
   else:
        print(f"{key}: {type(value).__name__} : {value}")
  print(f"Total Number of segments: {len(task.video_embedding.segments)}")

67896e59c6dde497bd15fc8b
start_offset_sec: float : 0.0
end_offset_sec: float : 6.0
embedding_scope: str : clip
embeddings_float: list of size 1024 (truncated to 5 items): [0.030361895, 0.008698823, -0.0048321243, -0.019013105, -0.011488311] 
Total Number of segments: 37
67896ea6c6dde497bd15fc91
start_offset_sec: float : 0.0
end_offset_sec: float : 6.0
embedding_scope: str : clip
embeddings_float: list of size 1024 (truncated to 5 items): [0.024328815, -0.0035867887, 0.016065866, 0.02501548, 0.007778642] 
Total Number of segments: 242
67896f6ba761bcb96501c241
start_offset_sec: float : 0.0
end_offset_sec: float : 6.0
embedding_scope: str : clip
embeddings_float: list of size 1024 (truncated to 5 items): [0.04080625, 0.0086980555, 0.00096186635, -0.00607, -0.020250283] 
Total Number of segments: 110


#3. Deploy a Vespa Application

At this point, we are ready to deploy a Vespa Application. We have generated the attributes we needed on each video, as well as the embeddings.

## 3.1 Create an Application Package

The [application package](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.ApplicationPackage)
has all the Vespa configuration files -
create one from scratch:

The Vespa schema deployed as part of the package is called `videos`. All the fields are matching the output of the Twelvelabs Embed API above. Refer to the [Vespa documentation](https://docs.vespa.ai/en/reference/schema-reference.html) for more information on the schema specification.

We can first define the schema using pyvespa

In [ ]:
videos_schema = Schema(
            name="videos",
            document=Document(
                fields=[
                    Field(
                        name="video_url",
                        type="string",
                        indexing=["summary"]
                    ),
                    Field(
                        name="title",
                        type="string",
                        indexing=["index", "summary"],
                        match=["text"],
                        index="enable-bm25",
                    ),
                    Field(
                        name="keywords",
                        type="string",
                        indexing=["index", "summary"],
                        match=["text"],
                        index="enable-bm25",
                    ),

                    Field(
                        name="video_summary",
                        type="string",
                        indexing=["index", "summary"],
                        match=["text"],
                        index="enable-bm25",

                    ),
                    Field(
                        name="embedding_scope",
                        type="string",
                        indexing=["attribute", "summary"]
                    ),
                    Field(
                        name="start_offset_sec",
                        type="array<float>",
                        indexing=["attribute","summary"]
                    ),
                    Field(
                        name="end_offset_sec",
                        type="array<float>",
                        indexing=["attribute", "summary"]
                    ),
                    Field(
                        name="embeddings",
                        type="tensor<float>(p{},x[1024])",
                        indexing=["index", "attribute"],
                        ann=HNSW(distance_metric="angular"),
                    ),


                ]
            ),
        )

fieldsets=[
        FieldSet(
            name="default",
            fields=["title", "keywords", "video_summary"],
        ),
    ],

mapfunctions = [
            Function(
                      name="similarities",
                      expression="""
                      sum(
                          query(q) * attribute(embeddings), x
                          )
                      """,
                    ),
            Function(name="bm25_score", expression="bm25(title) + bm25(keywords) + bm25(video_summary)")

]

semantic_rankprofile = RankProfile(
    name="hybrid",
    inputs=[("query(q)", "tensor<float>(x[1024])")],
    first_phase="bm25_score",
    second_phase=SecondPhaseRanking(expression="closeness(field, embeddings)", rerank_count=10),
    match_features=["closest(embeddings)"],
    summary_features=["similarities"],
    functions=mapfunctions
)

videos_schema.add_rank_profile(semantic_rankprofile)

We can now create the package based on the previous schema

In [ ]:
# Create the Vespa application package
package = ApplicationPackage(
                            name=application,
                            schema=[videos_schema])

## 3.2 Deploy the Application Package

The app is now defined and ready to deploy to Vespa Cloud.

Deploy `package` to Vespa Cloud, by creating an instance of
[VespaCloud](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaCloud):

In [ ]:
vespa_cloud = VespaCloud(
    tenant=tenant_name,
    application=application,
    application_package=package,
)

Setting application...
Running: vespa config set application vespa-presales.twelvelabsmv
Setting target cloud...
Running: vespa config set target cloud

No api-key found for control plane access. Using access token.
Checking for access token in auth.json...
Access token expired. Please re-authenticate.
Your Device Confirmation code is: QLMD-HVWG
Automatically open confirmation page in your default browser? [Y/n] 

Opened link in your browser: https://login.console.vespa-cloud.com/activate?user_code=QLMD-HVWG
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://login.console.vespa-cloud.com/activate?user_code=QLMD-HVWG'
Couldn't open the URL, please do it manually
Waiting for login to complete in browser ... done
Success: Logged in
 auth.json

In [ ]:
app = vespa_cloud.deploy()

Deployment started in run 47 of dev-aws-us-east-1c for vespa-presales.twelvelabsmv. This may take a few minutes the first time.
INFO    [20:47:04]  Deploying platform version 8.464.10 and application dev build 46 for dev-aws-us-east-1c of default ...
INFO    [20:47:04]  Using CA signed certificate version 1
INFO    [20:47:05]  Using 1 nodes in container cluster 'twelvelabsmv_container'
INFO    [20:47:06]  Using 1 nodes in container cluster 'twelvelabsmv_container'
INFO    [20:47:09]  Session 739 for tenant 'vespa-presales' prepared and activated.
INFO    [20:47:10]  ######## Details for all nodes ########
INFO    [20:47:10]  h97482a.dev.us-east-1c.aws.vespa-cloud.net: expected to be UP
INFO    [20:47:10]  --- platform vespa/cloud-tenant-rhel8:8.464.10
INFO    [20:47:10]  --- storagenode on port 19102 has config generation 737, wanted is 739
INFO    [20:47:10]  --- searchnode on port 19107 has config generation 739, wanted is 739
INFO    [20:47:10]  --- distributor on port 19111 has con

## 3.3 Feed the Vespa Application

The `vespa_feed` feed format for `pyvespa` expects a dict with the keys `id` and `fields`:

`{ "id": "vespa-document-id", "fields": {"vespa_field": "vespa-field-value"}}`

For the id, we will use a md5 hash of the video url.

The video embedding output segments are added to the `fields` in `vespa_feed`.

In [ ]:
# Initialize a list to store Vespa feed documents
vespa_feed = []

#Need to reverse VIDEO_URLS as keywords/summaries generated in reverse order
VIDEO_URLs.reverse()

# Iterate through each task and corresponding metadata
for i, task in enumerate(tasks):
    video_url = VIDEO_URLs[i]
    title = titles[i]
    keywords = keywords_array[i]
    summary = summaries[i]

    start_offsets = []  # Reset for each video
    end_offsets = []  # Reset for each video
    embeddings = {}  # Reset for each video

    # Iterate through the video embedding segments
    for index, segment in enumerate(task.video_embedding.segments):
        # Append start and end offsets as floats
        start_offsets.append(float(segment.start_offset_sec))
        end_offsets.append(float(segment.end_offset_sec))

        # Add embedding to a multi-dimensional dictionary with index as the key
        embeddings[str(index)] = list(map(float, segment.embeddings_float))

    # Create Vespa document for each task
    for segment in task.video_embedding.segments:
        start_offset_sec = segment.start_offset_sec
        end_offset_sec = segment.end_offset_sec
        embedding = list(map(float, segment.embeddings_float))

        # Create a unique ID by hashing the URL and segment index
        id_hash = hashlib.md5(f"{video_url}_{index}".encode()).hexdigest()

        document = {
            "id": id_hash,
            "fields": {
                "video_url": video_url,
                "title": title,
                "keywords": keywords,
                "video_summary": summary,
                "embedding_scope": segment.embedding_scope,
                "start_offset_sec": start_offsets,
                "end_offset_sec": end_offsets,
                "embeddings": embeddings
            },
        }
    vespa_feed.append(document)


We can quickly validate the number of the number of documents created (one for each video), and visually check the first record.

In [ ]:
# Print Vespa feed size and an example
print(f"Total documents created: {len(vespa_feed)}")

Total documents created: 3


In [ ]:
# The positional index of the document
i=0

# Iterate through the first 3 embeddings in vespa_feed
for i in range(min(3, len(vespa_feed))):  # Ensure we don't exceed the length of vespa_feed
    # Limit the embedding to the first 3 keys and first 5 values for each key
    embedding = vespa_feed[i]["fields"]["embeddings"]
    embedding_sample = {
        key: values[:3] for key, values in list(embedding.items())[:3]
    }

# Beautify and print the first document with only the first 5 embedding values
pretty_json = json.dumps({
    "id": vespa_feed[i]["id"],
    "fields": {
        "video_url": vespa_feed[i]["fields"]["video_url"],
        "title": vespa_feed[i]["fields"]["title"],
        "keywords": vespa_feed[i]["fields"]["keywords"],
        "video_summary": vespa_feed[i]["fields"]["video_summary"],
        "embedding_scope": vespa_feed[i]["fields"]["embedding_scope"],
        "start_offset_sec": vespa_feed[i]["fields"]["start_offset_sec"][:3],
        "end_offset_sec": vespa_feed[i]["fields"]["end_offset_sec"][:3],
        "embedding": embedding_sample
    }
}, indent=4)

print(pretty_json)

{
    "id": "0b1fc68a17391fb58102a539ed290d27",
    "fields": {
        "video_url": "https://ia801503.us.archive.org/27/items/hide-and-seek-with-giant-jenny/HnVideoEditor_2022_10_29_205557707.ia.mp4",
        "title": "Hide and Seek with Giant Jenny",
        "keywords": "children's animation, anthropomorphic animals, imaginative play, emotional journey, colorful cartoon",
        "video_summary": "The video is a delightful compilation from a children's animated series featuring anthropomorphic animal characters, showcasing their daily lives and adventures. The animation is vibrant and whimsical, with exaggerated expressions and movements that captivate young audiences.\nThe video begins with a lively scene where the characters make a grand entrance on a red carpet, setting a playful tone. This is followed by various snippets of their daily activities, including sleeping, playing, and watching TV. The characters engage in imaginative play with toys, navigate through a bustling citysca

Now we can feed to Vespa using `feed_iterable` which accepts any `Iterable` and an optional callback function where we can
check the outcome of each operation.

In [ ]:
def callback(response: VespaResponse, id: str):
    if not response.is_successful():
        print(
            f"Failed to feed document {id} with status code {response.status_code}: Reason {response.get_json()}"
        )


# Feed data into Vespa synchronously
app.feed_iterable(vespa_feed, schema="videos", callback=callback)

# 4. Performing search on the videos


## 4.1 Performing a hybrid search on the video

As an example query, we will retrieve all the chunks which shows Santa Claus on his sleigh. The first step is to generate a text embedding for `Santa Claus on his sleigh` using the `Marengo-retrieval-2.7` model.

In [ ]:
client = TwelveLabs(api_key=TL_API_KEY)
user_query = "Santa Claus on his sleigh"

res = client.embed.create(
  model_name="Marengo-retrieval-2.7",
  text=user_query,
)

print("Created a text embedding")
print(f" Model: {res.model_name}")
if res.text_embedding is not None and res.text_embedding.segments is not None:
        q_embedding=res.text_embedding.segments[0].embeddings_float
        print(f" Embedding Dimension: {len(q_embedding)}")
        print(q_embedding[5:])



Created a text embedding
 Model: Marengo-retrieval-2.7
 Embedding Dimension: 1024
[-0.0076904297, -0.0043029785, 0.03564453, -0.06933594, -0.0625, -0.0020751953, -0.00020217896, -0.015319824, 0.017822266, 0.050048828, 0.006713867, 0.02734375, -0.052490234, 0.028686523, -0.03515625, 0.0057678223, 0.02734375, 0.018188477, 0.043945312, -0.030883789, 0.03955078, 0.06933594, 0.025146484, -0.057861328, 0.010925293, -0.0025177002, -0.04638672, 0.020629883, 0.014892578, -0.010009766, 0.041259766, -0.0057373047, -0.021728516, 0.000957489, -0.049072266, 0.011474609, 0.015380859, 0.03930664, -0.010009766, -0.0020446777, -0.03491211, -0.005218506, -0.01159668, -0.028076172, -0.0054626465, 0.024414062, 0.033203125, 0.021240234, 0.0018920898, 0.00074768066, 0.02722168, -0.07763672, -0.030273438, 0.033447266, -0.00970459, 0.008117676, -0.03881836, 0.0030670166, 0.0087890625, 0.023925781, 0.05078125, 0.015258789, 0.0032196045, 0.031982422, 0.010437012, 0.03149414, 0.030273438, 0.033203125, -0.02062988

The following uses dense vector representations of the query embedding obtained previously and document and matching is performed and accelerated by Vespa's support for
[approximate nearest neighbor search](https://docs.vespa.ai/en/approximate-nn-hnsw.html).

The output is limited to the top 1 hit, as we only have a sample of 3 videos. The top hit returned was based on a hybrid ranking based on a bm25 ranking based on a lexical search on the text, keywords and summary of the video, performed as a first phase, and similarity search on the embeddings.

We can see as part of the `match-features`, the segment 212 in the video was the one providing the highest match.

We also calculate the similarities as part of the `summary-features` for the rest of the segments so we can look for top N segments within a video, optionally.


In [ ]:
with app.syncio(connections=1) as session:
    response: VespaQueryResponse = session.query(
        yql="select * from videos where userQuery() OR ({targetHits:100}nearestNeighbor(embeddings,q))",
        query=user_query,
        ranking="hybrid",
        hits=1,
        body={"input.query(q)": q_embedding}
    )
    assert response.is_successful()

for hit in response.hits:
    print(json.dumps(hit,indent=4))

response.get_json()

{
    "id": "id:videos:videos::13bcb994b389c9d925993e611877e40b",
    "relevance": 0.47900226064497403,
    "source": "twelvelabsmv_content",
    "fields": {
        "matchfeatures": {
            "closest(embeddings)": {
                "type": "tensor<float>(p{})",
                "cells": {
                    "212": 1.0
                }
            }
        },
        "sddocname": "videos",
        "documentid": "id:videos:videos::13bcb994b389c9d925993e611877e40b",
        "video_url": "https://ia601401.us.archive.org/1/items/twas-the-night-before-christmas-1974-full-movie-freedownloadvideo.net/twas-the-night-before-christmas-1974-full-movie-freedownloadvideo.net.mp4",
        "title": "Twas the night before Christmas",
        "keywords": "Christmas, Santa Claus, animated special, holiday magic, teamwork",
        "video_summary": "In Junctionville, Santa cancels Christmas after receiving an anonymous letter claiming he doesn\u2019t exist. Joshua Trundle and Father Mouse create 

{'root': {'id': 'toplevel',
  'relevance': 1.0,
  'fields': {'totalCount': 3},
  'coverage': {'coverage': 100,
   'documents': 3,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'children': [{'id': 'id:videos:videos::13bcb994b389c9d925993e611877e40b',
    'relevance': 0.47900226064497403,
    'source': 'twelvelabsmv_content',
    'fields': {'matchfeatures': {'closest(embeddings)': {'type': 'tensor<float>(p{})',
       'cells': {'212': 1.0}}},
     'sddocname': 'videos',
     'documentid': 'id:videos:videos::13bcb994b389c9d925993e611877e40b',
     'video_url': 'https://ia601401.us.archive.org/1/items/twas-the-night-before-christmas-1974-full-movie-freedownloadvideo.net/twas-the-night-before-christmas-1974-full-movie-freedownloadvideo.net.mp4',
     'title': 'Twas the night before Christmas',
     'keywords': 'Christmas, Santa Claus, animated special, holiday magic, teamwork',
     'video_summary': 'In Junctionville, Santa cancels Christmas after receiving an ano

In order to process the results above in a more consumable format and sort out the top N segments based on similarities, we can do this more conveniently in a pandas dataframe below:

In [ ]:
def get_top_n_similarity_matches(data, N=5):
    """
    Function to extract the top N similarity scores and their corresponding start and end offsets.

    Args:
    - data (dict): Input JSON-like structure containing similarities and offsets.
    - N (int): The number of top similarity scores to return.

    Returns:
    - pd.DataFrame: A DataFrame with the top N similarity scores and their corresponding offsets.
    """
    # Extract relevant fields
    similarities = data["fields"]["summaryfeatures"]["similarities"]["cells"]
    start_offset_sec = data["fields"]["start_offset_sec"]
    end_offset_sec = data["fields"]["end_offset_sec"]

    # Convert similarity scores to a list of tuples (index, similarity_score) and sort by similarity score
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

    # Extract top N similarity scores
    top_n_similarities = sorted_similarities[:N]

    # Prepare results
    results = []
    for index_str, score in top_n_similarities:
        index = int(index_str)
        if index < len(start_offset_sec):
            result = {
                "index": index,
                "similarity_score": score,
                "start_offset_sec": start_offset_sec[index],
                "end_offset_sec": end_offset_sec[index]
            }
        else:
            result = {
                "index": index,
                "similarity_score": score,
                "start_offset_sec": None,
                "end_offset_sec": None
            }
        results.append(result)

    # Convert results to a DataFrame
    df = pd.DataFrame(results)
    return df



In [ ]:
df_result=get_top_n_similarity_matches(response.hits[0], N=10)
df_result

,index,similarity_score,start_offset_sec,end_offset_sec
0,212,0.464550,1272.0,1278.0
1,230,0.440820,1380.0,1386.0
2,208,0.436963,1248.0,1254.0
3,210,0.435095,1260.0,1266.0
4,231,0.434616,1386.0,1392.0
5,211,0.431943,1266.0,1272.0
6,229,0.429756,1374.0,1380.0
7,209,0.427482,1254.0,1260.0
8,203,0.426361,1218.0,1224.0
9,207,0.415856,1242.0,1248.0


## 5. Review results (Optional)

We can review the results by spinning up a video player in the notebook and check the segments identified and judge by ourselves.

But, first we need to obtain the contiguous segments, add 3 seconds overlap in the consolidated segments and convert to MM:SS so we can quickly find the segments to watch in the player. Let's write a function that takes the response as an input and provides the consolidated segments to view in the player.

In [ ]:
def concatenate_contiguous_segments(df):
    """
    Function to concatenate contiguous segments based on their start and end offsets.
    Converts the concatenated segments to MM:SS format.

    Args:
    - df (pd.DataFrame): DataFrame with columns 'start_offset_sec' and 'end_offset_sec'.

    Returns:
    - List of tuples with concatenated segments in MM:SS format as (start_time, end_time).
    """
    if df.empty:
        return []

    # Sort by start_offset_sec for ordered processing
    df = df.sort_values(by="start_offset_sec").reset_index(drop=True)

    # Initialize the list to hold concatenated segments
    concatenated_segments = []

    # Initialize the first segment
    start = df.iloc[0]["start_offset_sec"]
    end = df.iloc[0]["end_offset_sec"]

    for i in range(1, len(df)):
        current_start = df.iloc[i]["start_offset_sec"]
        current_end = df.iloc[i]["end_offset_sec"]

        # Check if the current segment is contiguous with the previous one
        if current_start <= end:
            # Extend the segment if it is contiguous
            end = max(end, current_end)
        else:
            # Add the previous segment to the result list in MM:SS format
            concatenated_segments.append((convert_seconds_to_mmss(start-3), convert_seconds_to_mmss(end+3)))
            # Start a new segment
            start = current_start
            end = current_end

    # Add the final segment
    concatenated_segments.append((convert_seconds_to_mmss(start-3), convert_seconds_to_mmss(end+3)))

    return concatenated_segments


def convert_seconds_to_mmss(seconds):
    """
    Converts seconds to MM:SS format.

    Args:
    - seconds (float): Time in seconds.

    Returns:
    - str: Time in MM:SS format.
    """
    minutes = int(seconds // 60)
    seconds = int(seconds % 60)
    return f"{minutes:02}:{seconds:02}"



In [ ]:
concatenate_contiguous_segments(df_result)

[('20:15', '20:27'), ('20:39', '21:21'), ('22:51', '23:15')]

We can now spin-up the player and review the segments of interest.

In [ ]:
from IPython.display import HTML

video_url = "https://ia601401.us.archive.org/1/items/twas-the-night-before-christmas-1974-full-movie-freedownloadvideo.net/twas-the-night-before-christmas-1974-full-movie-freedownloadvideo.net.mp4"

# Create an HTML video player
video_player = f"""
<video width="640" height="480" controls>
  <source src="{video_url}" type="video/mp4">
  Your browser does not support the video tag.
</video>
"""

# Display the video player
HTML(video_player)